<a href="https://colab.research.google.com/github/abecode/teaching_reviews/blob/main/notebook/annotation_analysis_20240617.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/abecode/teaching_reviews.git

Cloning into 'teaching_reviews'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 80 (delta 18), reused 67 (delta 6), pack-reused 0
Receiving objects: 100% (80/80), 336.13 KiB | 2.71 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
!cp teaching_reviews/prodigy_data/prodigy.sentence-level.20240605.db prodigy.db

for info on sqlite python library, see https://docs.python.org/3/library/sqlite3.html

```
# This is formatted as code
```



In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('prodigy.db')

In [ ]:
abe_annotations = con.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                 from dataset, example, link
                                 where dataset.id = link.dataset_id
                                   and link.example_id = example.id
                                   and dataset.name = 'teaching_reviews_sentences-abe';""").fetchall()

In [ ]:
len(abe_annotations)

725

In [ ]:
mengyuan_annotations = con.execute("""select dataset.id, dataset.name, dataset.created, dataset.meta, dataset.session,
                                        example.id, example.input_hash, example.task_hash, example.content
                                      from dataset, example, link
                                      where dataset.id = link.dataset_id
                                        and link.example_id = example.id
                                        and dataset.name = 'teaching_reviews_sentences-mengyuan';""").fetchall()

In [ ]:
len(mengyuan_annotations)

725

make sure that the results add up: there should be 725 inputs for each annotator, 1450 unique tasks and 725 unique inputs



In [ ]:
input_hashes = [];
task_hashes = [];
for annotation in abe_annotations:
  input_hashes.append(annotation[5])
  task_hashes.append(annotation[6])
for annotation in mengyuan_annotations:
  input_hashes.append(annotation[5])
  task_hashes.append(annotation[6])

In [ ]:
print(len(input_hashes))
print(len(task_hashes))
print(len(set(input_hashes)))
print(len(set(task_hashes)))

1450
1450
1450
725


💡 interesting, actually it is the task_hash that is unique for the input

In [ ]:
import pandas as pd

In [ ]:
import json

https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [ ]:
abe_anno_df = pd.DataFrame({"annotator": ["abe" for x in abe_annotations],
                           "input_hash": [x[5] for x in abe_annotations],
                           "task_hash": [x[6] for x in abe_annotations],
                           #"content": [x[8] for x in abe_annotations],
                           "text": [json.loads(x[8])['text'] for x in abe_annotations],
                           "annotation":  [json.loads(x[8])['accept'] for x in abe_annotations]})

In [ ]:
abe_anno_df.head()

,annotator,input_hash,task_hash,text,annotation
0,abe,726,-1514685370,"Abe is a practical, passionate, and thoughtful...",[POSITIVE]
1,abe,727,1097655643,My only feedback is that the class seems to be...,"[MIXED, SUGGESTION]"
2,abe,728,1814679939,Professor Abe was an absolute blessing to have...,[POSITIVE]
3,abe,729,586397455,"He was approachable, funny, and so helpful in ...",[POSITIVE]
4,abe,730,-232063929,Great and understanding instructor who handled...,[POSITIVE]


In [ ]:
mengyuan_anno_df = pd.DataFrame({"annotator": ["mengyuan" for x in mengyuan_annotations],
                                 "input_hash": [x[5] for x in mengyuan_annotations],
                                 "task_hash": [x[6] for x in mengyuan_annotations],
                                 #"content": [x[8] for x in mengyuan_annotations],
                                 "text": [json.loads(x[8])['text'] for x in mengyuan_annotations],
                                 "annotation":  [json.loads(x[8])['accept'] for x in mengyuan_annotations]
                                 })

In [ ]:
mengyuan_anno_df.head()

,annotator,input_hash,task_hash,text,annotation
0,mengyuan,1,-1514685370,"Abe is a practical, passionate, and thoughtful...",[POSITIVE]
1,mengyuan,2,1097655643,My only feedback is that the class seems to be...,[SUGGESTION]
2,mengyuan,3,1814679939,Professor Abe was an absolute blessing to have...,[POSITIVE]
3,mengyuan,4,586397455,"He was approachable, funny, and so helpful in ...",[POSITIVE]
4,mengyuan,5,-232063929,Great and understanding instructor who handled...,[POSITIVE]


In [ ]:
all_anno_df = pd.concat([abe_anno_df, mengyuan_anno_df])

In [ ]:
anno_df = all_anno_df.pivot(index=['task_hash', 'text'], columns=['annotator'], values=['annotation'])
print(anno_df.shape, anno_df.columns) # here columns are a multi-index, don't understand
anno_df.columns = anno_df.columns.droplevel(0) # this fixes the multi index column
print(anno_df.shape, anno_df.columns) # but the index is still multi-level
anno_df.reset_index(inplace=True). # this fixes the multi-level index
print(anno_df.shape, anno_df.columns) # how the df is 725x4
anno_df.head()

(725, 2) MultiIndex([('annotation',      'abe'),
            ('annotation', 'mengyuan')],
           names=[None, 'annotator'])
(725, 2) Index(['abe', 'mengyuan'], dtype='object', name='annotator')
(725, 4) Index(['task_hash', 'text', 'abe', 'mengyuan'], dtype='object', name='annotator')


annotator,task_hash,text,abe,mengyuan
0,-2142112799,Using more plain language to explain difficult...,[SUGGESTION],[SUGGESTION]
1,-2140549093,Sometime reading the book is hard and correlat...,[NEGATIVE],[NEGATIVE]
2,-2119156260,A smallish assignment that checks the box for ...,[NEUTRAL],[UNSURE]
3,-2107663911,Maybe it's part of being in the Saturday class...,[NEGATIVE],[NEGATIVE]
4,-2107420223,I do think that final project helps tie things...,[POSITIVE],[NEGATIVE]


In [ ]:
anno_df

annotator,abe,mengyuan
text,,
"Using more plain language to explain difficult concepts would be helpful, as well as more examples and writing more on the board to provide visual examples.",[SUGGESTION],[SUGGESTION]
Sometime reading the book is hard and correlate with R seems little challenging.,[NEGATIVE],[NEGATIVE]
"A smallish assignment that checks the box for ""hands on work"" 4.",[NEUTRAL],[UNSURE]
"Maybe it's part of being in the Saturday class but, especially by the afternoon, it was hard to just sit and listen and look at lecture slides.",[NEGATIVE],[NEGATIVE]
I do think that final project helps tie things together and gives a better meaning to the course and why I took it.,[POSITIVE],[NEGATIVE]
...,...,...
I feel that I could have learned more about Tableau by reading a book and working through exercises on my own.,[NEGATIVE],[NEGATIVE]
The lectures were too long.,[NEGATIVE],[NEGATIVE]
I would have liked more Oracle assignments and hands-on activities (instead of demos by Prof Abe).,[SUGGESTION],"[NEGATIVE, SUGGESTION]"


In [ ]:
anno_df['agreement'] = anno_df['abe'] == anno_df['mengyuan']
anno_df.head()

annotator,task_hash,text,abe,mengyuan,agreement
0,-2142112799,Using more plain language to explain difficult...,[SUGGESTION],[SUGGESTION],True
1,-2140549093,Sometime reading the book is hard and correlat...,[NEGATIVE],[NEGATIVE],True
2,-2119156260,A smallish assignment that checks the box for ...,[NEUTRAL],[UNSURE],False
3,-2107663911,Maybe it's part of being in the Saturday class...,[NEGATIVE],[NEGATIVE],True
4,-2107420223,I do think that final project helps tie things...,[POSITIVE],[NEGATIVE],False


In [ ]:
anno_df.agreement.value_counts()

agreement
True     513
False    212
Name: count, dtype: int64

In [ ]:
anno_df.query('agreement == False')

annotator,task_hash,text,abe,mengyuan,agreement
2,-2119156260,A smallish assignment that checks the box for ...,[NEUTRAL],[UNSURE],False
4,-2107420223,I do think that final project helps tie things...,[POSITIVE],[NEGATIVE],False
7,-2098281465,In the future he should really just tell peopl...,[SUGGESTION],"[NEGATIVE, SUGGESTION]",False
12,-2066565131,The decision to have us prerecord the project ...,[POSITIVE],[NEGATIVE],False
13,-2062770261,Take this class from Professor Ann instead.,[NEUTRAL],[POSITIVE],False
...,...,...,...,...,...
711,2098085936,I am also taking the stats class in addition t...,[NEUTRAL],[UNSURE],False
712,2098814447,I have no doubt in Prof. Kazemzadeh's expertis...,"[POSITIVE, NEGATIVE, NEUTRAL, MIXED, UNSURE]",[NEUTRAL],False
716,2110241957,I would rather work with my peers on a project...,"[NEUTRAL, UNSURE, SUGGESTION]",[SUGGESTION],False
722,2123931757,I would have liked more Oracle assignments and...,[SUGGESTION],"[NEGATIVE, SUGGESTION]",False


In [ ]:
anno_df.query('agreement != True').to_excel('disagreements.xlsx')